In [1]:
# Impor tensorflow and numpy
import tensorflow as tf
import numpy as np
import math as m
import time
# For validation
from sklearn.metrics import confusion_matrix
import itertools
import tensorpack.dataflow
# For plotting
from matplotlib import pyplot as plt

# Add the library to the system path
import os,sys
os.environ['CUDA_VISIBLE_DEVICES']='0'
se2cnn_source =  os.path.join(os.getcwd(),'..')
if se2cnn_source not in sys.path:
    sys.path.append(se2cnn_source)

# Import the library
import se2cnn.layers

/home/localadmin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorpack import logger, QueueInput
from tensorpack.models import *
from tensorpack.callbacks import *
from tensorpack.train import (
    TrainConfig, SyncMultiGPUTrainerReplicated, launch_train_with_config)
from tensorpack.dataflow import FakeData
from tensorpack.tfutils import argscope, get_model_loader
from tensorpack.utils.gpu import get_num_gpu
from imagenet_utils import (
    fbresnet_augmentor, get_imagenet_dataflow, ImageNetModel,
    eval_on_ILSVRC12)
from tensorpack import *

In [3]:
# Xavier's/He-Rang-Zhen-Sun initialization for layers that are followed ReLU
def weight_initializer(n_in, n_out):
    return tf.random_normal_initializer(mean=0.0, stddev=m.sqrt(2.0 / (n_in))
    )

In [4]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [5]:
def size_of(tensor) :
    # Multiply elements one by one
    result = 1
    for x in tensor.get_shape().as_list():
         result = result * x 
    return result 

# load imagenet

In [6]:
Image_net_path = "/mnt/nas2/gx/ImageNet"
batch = 128

In [7]:
def get_data(name, batch):
    isTrain = name == 'train'
    augmentors = fbresnet_augmentor(isTrain)
    return get_imagenet_dataflow(
        Image_net_path, name, batch, augmentors)

***
# Part 3: Build a graph (design the G-CNN)

## Build a graph

In [8]:
graph = tf.Graph()
graph.as_default()
tf.reset_default_graph()

In [9]:
Ntheta = 1 # Kernel size in angular direction
Nxy=5       # Kernel size in spatial direction
Nc = 4      # Number of channels in the initial layer

In [10]:
inputs_ph = tf.placeholder( dtype = tf.float32, shape = [None,224,224,3] )
labels_ph = tf.placeholder( dtype = tf.int32, shape = [None,] )

In [11]:
tensor_in = inputs_ph
Nc_in = 3

In [12]:
kernels={}

In [13]:
with tf.variable_scope("Layer_{}".format(1)) as _scope:
    ## Settings
    Nc_out = Nc

    ## Perform lifting convolution
    # The kernels used in the lifting layer
    kernels_raw = tf.get_variable(
                        'kernel', 
                        [Nxy,Nxy,Nc_in,Nc_out],
                        initializer=weight_initializer(Nxy*Nxy*Nc_in,Nc_out))
    tf.add_to_collection('raw_kernels', kernels_raw)
    bias = tf.get_variable( # Same bias for all orientations
                        "bias",
                        [1, 1, 1, 1, Nc_out], 
                        initializer=tf.constant_initializer(value=0.01))
    # Lifting layer
    tensor_out, kernels_formatted = se2cnn.layers.z2_se2n(
                            input_tensor = tensor_in,
                            kernel = kernels_raw,
                            orientations_nb = Ntheta)
    # Add bias
    tensor_out = tensor_out + bias
    
    
    ## Perform (spatial) max-pooling
    tensor_out = se2cnn.layers.spatial_max_pool( input_tensor=tensor_out, nbOrientations=Ntheta)
    
    ## Apply ReLU
    tensor_out = tf.nn.relu(tensor_out)

    ## Prepare for the next layer
    tensor_in = tensor_out
    Nc_in = Nc_out
    
    ## Save kernels for inspection
    kernels[_scope.name] = kernels_formatted

Z2-SE2N BASE KERNEL SHAPE: (5, 5, 3, 4)
Z2-SE2N ROTATED KERNEL SET SHAPE: (1, 5, 5, 3, 4)
OUTPUT SE2N ACTIVATIONS SHAPE: (?, ?, ?, 1, 4)


In [14]:
tensor_in.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(1), Dimension(4)])

# Layer 2: SE2-conv, max-pool, relu

In [15]:
with tf.variable_scope("Layer_{}".format(2)) as _scope:
    ## Settings
    Nc_out = Nc

    ## Perform group convolution
    # The kernels used in the group convolution layer
    kernels_raw = tf.get_variable(
                        'kernel', 
                        [Nxy,Nxy,Ntheta,Nc_in,Nc_out],
                        initializer=weight_initializer(Nxy*Nxy*Ntheta*Nc_in,Nc_out))
    tf.add_to_collection('raw_kernels', kernels_raw)
    bias = tf.get_variable( # Same bias for all orientations
                        "bias",
                        [1, 1, 1, 1, Nc_out], 
                        initializer=tf.constant_initializer(value=0.01))
    # The group convolution layer
    tensor_out, kernels_formatted = se2cnn.layers.se2n_se2n(
                            input_tensor = tensor_in,
                            kernel = kernels_raw)
    tensor_out = tensor_out + bias
    
    ## Perform max-pooling
    tensor_out = se2cnn.layers.spatial_max_pool( input_tensor=tensor_out, nbOrientations=Ntheta)
    
    ## Apply ReLU
    tensor_out = tf.nn.relu(tensor_out)

    ## Prepare for the next layer
    tensor_in = tensor_out
    Nc_in = Nc_out
    
    ## Save kernels for inspection
    kernels[_scope.name] = kernels_formatted

SE2N-SE2N BASE KERNEL SHAPE: (5, 5, 1, 4, 4)
SE2N-SE2N ROTATED KERNEL SET SHAPE: (1, 5, 5, 1, 4, 4)
OUTPUT SE2N ACTIVATIONS SHAPE: (?, ?, ?, 1, 4)


In [16]:
with tf.variable_scope("Layer_{}".format(3)) as _scope:
    ## Settings
    Nc_out = 2*Nc
    Nxy = 6
    ## Perform group convolution
    # The kernels used in the group convolution layer
    kernels_raw = tf.get_variable(
                        'kernel', 
                        [Nxy,Nxy,Ntheta,Nc_in,Nc_out],
                        initializer=weight_initializer(Nxy*Nxy*Ntheta*Nc_in,Nc_out))
    tf.add_to_collection('raw_kernels', kernels_raw)
    bias = tf.get_variable( # Same bias for all orientations
                        "bias",
                        [1, 1, 1, 1, Nc_out], 
                        initializer=tf.constant_initializer(value=0.01))
    # The group convolution layer
    tensor_out, kernels_formatted = se2cnn.layers.se2n_se2n(
                            input_tensor = tensor_in,
                            kernel = kernels_raw)
    tensor_out = tensor_out + bias
    
    ## Perform max-pooling
    tensor_out = se2cnn.layers.spatial_max_pool( input_tensor=tensor_out, nbOrientations=Ntheta)
    
    ## Apply ReLU
    tensor_out = tf.nn.relu(tensor_out)

    ## Prepare for the next layer
    tensor_in = tensor_out
    Nc_in = Nc_out
    
    ## Save kernels for inspection
    kernels[_scope.name] = kernels_formatted

SE2N-SE2N BASE KERNEL SHAPE: (6, 6, 1, 4, 8)
SE2N-SE2N ROTATED KERNEL SET SHAPE: (1, 6, 6, 1, 4, 8)
OUTPUT SE2N ACTIVATIONS SHAPE: (?, ?, ?, 1, 8)


In [17]:
tensor_in.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(1), Dimension(8)])

In [18]:
with tf.variable_scope("Layer_{}".format(4)) as _scope:
    ## Settings
    Nc_out = 2*Nc
    Nxy = 5
    ## Perform group convolution
    # The kernels used in the group convolution layer
    kernels_raw = tf.get_variable(
                        'kernel', 
                        [Nxy,Nxy,Ntheta,Nc_in,Nc_out],
                        initializer=weight_initializer(Nxy*Nxy*Ntheta*Nc_in,Nc_out))
    tf.add_to_collection('raw_kernels', kernels_raw)
    bias = tf.get_variable( # Same bias for all orientations
                        "bias",
                        [1, 1, 1, 1, Nc_out], 
                        initializer=tf.constant_initializer(value=0.01))
    # The group convolution layer
    tensor_out, kernels_formatted = se2cnn.layers.se2n_se2n(
                            input_tensor = tensor_in,
                            kernel = kernels_raw)
    tensor_out = tensor_out + bias
    
    ## Perform max-pooling
    tensor_out = se2cnn.layers.spatial_max_pool( input_tensor=tensor_out, nbOrientations=Ntheta)
    
    ## Apply ReLU
    tensor_out = tf.nn.relu(tensor_out)

    ## Prepare for the next layer
    tensor_in = tensor_out
    Nc_in = Nc_out
    
    ## Save kernels for inspection
    kernels[_scope.name] = kernels_formatted

SE2N-SE2N BASE KERNEL SHAPE: (5, 5, 1, 8, 8)
SE2N-SE2N ROTATED KERNEL SET SHAPE: (1, 5, 5, 1, 8, 8)
OUTPUT SE2N ACTIVATIONS SHAPE: (?, ?, ?, 1, 8)


In [19]:
tensor_in.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(1), Dimension(8)])

In [20]:
with tf.variable_scope("Layer_{}".format(5)) as _scope:
    ## Settings
    Nc_out = 2*Nc
    Nxy = 5
    ## Perform group convolution
    # The kernels used in the group convolution layer
    kernels_raw = tf.get_variable(
                        'kernel', 
                        [Nxy,Nxy,Ntheta,Nc_in,Nc_out],
                        initializer=weight_initializer(Nxy*Nxy*Ntheta*Nc_in,Nc_out))
    tf.add_to_collection('raw_kernels', kernels_raw)
    bias = tf.get_variable( # Same bias for all orientations
                        "bias",
                        [1, 1, 1, 1, Nc_out], 
                        initializer=tf.constant_initializer(value=0.01))
    # The group convolution layer
    tensor_out, kernels_formatted = se2cnn.layers.se2n_se2n(
                            input_tensor = tensor_in,
                            kernel = kernels_raw)
    tensor_out = tensor_out + bias
    
    ## Perform max-pooling
    tensor_out = se2cnn.layers.spatial_max_pool( input_tensor=tensor_out, nbOrientations=Ntheta)
    
    ## Apply ReLU
    tensor_out = tf.nn.relu(tensor_out)

    ## Prepare for the next layer
    tensor_in = tensor_out
    Nc_in = Nc_out
    
    ## Save kernels for inspection
    kernels[_scope.name] = kernels_formatted

SE2N-SE2N BASE KERNEL SHAPE: (5, 5, 1, 8, 8)
SE2N-SE2N ROTATED KERNEL SET SHAPE: (1, 5, 5, 1, 8, 8)
OUTPUT SE2N ACTIVATIONS SHAPE: (?, ?, ?, 1, 8)


In [21]:
tensor_in.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(1), Dimension(8)])

In [22]:
# Concatenate the orientation and channel dimension
tensor_in = tf.concat([tensor_in[:,:,:,i,:] for i in range(Ntheta)],3)
Nc_in = tensor_in.get_shape().as_list()[-1]

# 2D convolution layer
with tf.variable_scope("Layer_{}".format(6)) as _scope:
    ## Settings
    Nc_out = 4*Nc
    Nxy = 3
    ## Perform group convolution
    # The kernels used in the group convolution layer
    kernels_raw = tf.get_variable(
                        'kernel', 
                        [Nxy,Nxy,Nc_in,Nc_out],
                        initializer=weight_initializer(Nxy*Nxy*Nc_in,Nc_out))
    tf.add_to_collection('raw_kernels', kernels_raw)
    bias = tf.get_variable( # Same bias for all orientations
                        "bias",
                        [1, 1, 1, Nc_out], 
                        initializer=tf.constant_initializer(value=0.01))
    # Convolution layer
    tensor_out = tf.nn.conv2d(
                        input = tensor_in,
                        filter=kernels_raw,
                        strides=[1, 1, 1, 1],
                        padding="VALID")
    tensor_out = tensor_out + bias
    
    ## Apply ReLU
    tensor_out = tf.nn.relu(tensor_out)

    ## Prepare for the next layer
    tensor_in = tensor_out
    Nc_in = Nc_out
    
    ## Save kernels for inspection
    kernels[_scope.name] = kernels_raw

In [23]:
tensor_in.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(16)])

In [24]:
# 2D convolution layer
with tf.variable_scope("Layer_{}".format(7)) as _scope:
    ## Settings
    Nc_out = 128

    ## Perform group convolution
    # The kernels used in the group convolution layer
    kernels_raw = tf.get_variable(
                        'kernel', 
                        [1,1,Nc_in,Nc_out],
                        initializer=weight_initializer(1*1*Nc_in,Nc_out))
    tf.add_to_collection('raw_kernels', kernels_raw)
    bias = tf.get_variable( # Same bias for all orientations
                        "bias",
                        [1, 1, 1, Nc_out], 
                        initializer=tf.constant_initializer(value=0.01))
    # Convolution layer
    tensor_out = tf.nn.conv2d(
                        input = tensor_in,
                        filter=kernels_raw,
                        strides=[1, 1, 1, 1],
                        padding="VALID")
    tensor_out = tensor_out + bias
    
    ## Apply ReLU
    tensor_out = tf.nn.relu(tensor_out)

    ## Prepare for the next layer
    tensor_in = tensor_out
    Nc_in = Nc_out
    
    ## Save kernels for inspection
    kernels[_scope.name] = kernels_raw

In [25]:
tensor_in.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(128)])

In [26]:
with tf.variable_scope("Layer_{}".format(8)) as _scope:
    ## Settings
    Nc_out = 1000

    ## Perform group convolution
    # The kernels used in the group convolution layer
    kernels_raw = tf.get_variable(
                        'kernel', 
                        [1,1,Nc_in,Nc_out],
                        initializer=weight_initializer(1*1*Nc_in,Nc_out))
    tf.add_to_collection('raw_kernels', kernels_raw)
    bias = tf.get_variable( # Same bias for all orientations
                        "bias",
                        [1, 1, 1, Nc_out], 
                        initializer=tf.constant_initializer(value=0.01))

    
    ## Convolution layer
    tensor_out = tf.nn.conv2d(
                        input = tensor_in,
                        filter=kernels_raw,
                        strides=[1, 1, 1, 1],
                        padding="VALID")
    tensor_out = tensor_out + bias
    
    ## The output logits
    logits = tensor_out[:,0,0,:]
    predictions = tf.argmax(input=logits, axis=1)
    probabilities = tf.nn.softmax(logits)
    
    ## Save the kernels for later inspection
    kernels[_scope.name] = kernels_raw

In [27]:
logits.get_shape()

TensorShape([Dimension(None), Dimension(1000)])

## Define the loss and the optimizer

In [28]:
# Cross-entropy loss
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels_ph, logits=logits)

In [29]:
#-- Define the l2 loss 
weightDecay=5e-4
# Get the raw kernels
variables_wd = tf.get_collection('raw_kernels')
print('-----')
print('RAW kernel shapes:')
for v in variables_wd: print( "[{}]: {}, total nr of weights = {}".format(v.name, v.get_shape(), size_of(v)))
print('-----')
loss_l2 = weightDecay*sum([tf.nn.l2_loss(ker) for ker in variables_wd])

-----
RAW kernel shapes:
[Layer_1/kernel:0]: (5, 5, 3, 4), total nr of weights = 300
[Layer_2/kernel:0]: (5, 5, 1, 4, 4), total nr of weights = 400
[Layer_3/kernel:0]: (6, 6, 1, 4, 8), total nr of weights = 1152
[Layer_4/kernel:0]: (5, 5, 1, 8, 8), total nr of weights = 1600
[Layer_5/kernel:0]: (5, 5, 1, 8, 8), total nr of weights = 1600
[Layer_6/kernel:0]: (3, 3, 8, 16), total nr of weights = 1152
[Layer_7/kernel:0]: (1, 1, 16, 128), total nr of weights = 2048
[Layer_8/kernel:0]: (1, 1, 128, 1000), total nr of weights = 128000
-----


In [30]:
# Configure the Training Op (for TRAIN mode)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

train_op = optimizer.minimize(
    loss=loss + loss_l2,
    global_step=tf.train.get_global_step())

In [31]:
#-- Start the (GPU) session
initializer = tf.global_variables_initializer()
session = tf.Session(graph=tf.get_default_graph()) #-- Session created
session.run(initializer)

In [32]:
n_epochs=10

In [33]:
for epoch_nr in range(n_epochs):
    loss_sum = 0
    NItPerEpoch = 0
    df = get_data('train',batch)
    df.reset_state()
    generator = df.get_data()
    tStart = time.time()
    for myiter in generator:
        NItPerEpoch=NItPerEpoch+1
        feed_dict = {
                inputs_ph: myiter[0],
                labels_ph: myiter[1]
                }
        operators_output = session.run([ loss , train_op ], feed_dict)
        loss_sum += operators_output[0]
    loss_average = loss_sum/NItPerEpoch
    tElapsed = time.time() - tStart
    print('Epoch ' , epoch_nr , ' finished... Average loss = ' , round(loss_average,4) , ', time = ',round(tElapsed,4))

[0909 19:04:10 @fs.py:100] WRN Env var $TENSORPACK_DATASET not set, using /home/localadmin/tensorpack_data for datasets.
[0909 19:04:11 @imagenet_utils.py:104] WRN DataFlow may become the bottleneck when too few processes are used.
[0909 19:04:11 @parallel.py:291] [PrefetchDataZMQ] Will fork a dataflow more than one times. This assumes the datapoints are i.i.d.
[0909 19:04:11 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[0909 19:04:11 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[0909 19:04:11 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[0909 19:04:11 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[0909 19:04:11 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[0909 19:04:11 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guara

[0910 04:28:26 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[0910 04:28:26 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
Epoch  5  finished... Average loss =  5.9835 , time =  6900.4563
[0910 06:23:28 @imagenet_utils.py:104] WRN DataFlow may become the bottleneck when too few processes are used.
[0910 06:23:28 @parallel.py:291] [PrefetchDataZMQ] Will fork a dataflow more than one times. This assumes the datapoints are i.i.d.
PrefetchDataZMQ successfully cleaned-up.
[0910 06:23:28 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[0910 06:23:28 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[0910 06:23:28 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[0910 06:23:28 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[0910 06: